<a href="https://colab.research.google.com/github/NourEldin-Osama/Text-Simplification/blob/main/Notebooks/Voting_Ensample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.3 MB/s eta 0:00:00


In [2]:
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM)

from collections import Counter

In [3]:
checkpoint_1 = "husseinMoh/t5-small-finetuned-text-simplification"
checkpoint_2 = "husseinMoh/bart-base-finetuned-text-simplification"
checkpoint_3 = "husseinMoh/flan-t5-small-finetuned-text-simplification"

In [4]:
model1 = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_1)
model2 = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_2)
model3 = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_3)
models = [model1, model2, model3]

In [5]:
# Load tokenizers
tokenizer1 = AutoTokenizer.from_pretrained(checkpoint_1)
tokenizer2 = AutoTokenizer.from_pretrained(checkpoint_2)
tokenizer3 = AutoTokenizer.from_pretrained(checkpoint_3)
tokenizers = [tokenizer1,tokenizer2, tokenizer3]

In [6]:
def simplify(text, model, tokenizer):
    text = "" + text
    encoded_input = tokenizer(text, return_tensors="pt").input_ids
    outputs = model.generate(encoded_input)
    outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return outputs

def get_value(List, index, defualt_value=""):
    return List[index] if len(List) > index else defualt_value

def voting_ensemble(text, models, tokenizers, repeat=1):
    output_sentence = []
    simplifications = []
    for model, tokenizer in zip(models, tokenizers):
        simplification = simplify(text, model, tokenizer)
        simplifications.append(simplification)

    splitted_simplifications = [sentence.split() for sentence in simplifications]
    max_len_sentence = max([len(sentence) for sentence in splitted_simplifications])
    for index in range(max_len_sentence):
        words = [
                    get_value(splitted_simplifications[0], index),
                    get_value(splitted_simplifications[1], index),
                    get_value(splitted_simplifications[2], index),
                ]
        most_repeated_word, count = Counter(words).most_common(1)[0]
        # ignore empty string and if count less than 2
        if count > repeat and most_repeated_word:
            output_sentence.append(most_repeated_word)
    return " ".join(output_sentence)

In [7]:
# Test with an example text
input_text = "This is a complex sentence that needs to be simplified."
simplified_text = voting_ensemble(input_text, models, tokenizers, repeat=0)
print(simplified_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


The are many different kinds of Japanese characters, which are often written by


In [8]:
example_input_text = "The derivation of the equation is demonstrated through various mathematical"
example_simplified_text = voting_ensemble(example_input_text, models, tokenizers, repeat=0)
print(example_simplified_text)

Different methods of measuring different kinds of molecules are used by


In [9]:
voting_ensemble("Hi NourEldin", models, tokenizers)

'Hi NourEldin'